In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
np.random.seed(1)

In [3]:
import numpy as np
import pandas as pd
import scipy
from scipy import linalg
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from itertools import combinations
import pickle

In [4]:
ball = pd.read_csv('Half Ball Prediction Data.csv')
convex = pd.read_csv('Convex Prediction Data.csv')
freeform = pd.read_csv('Freeform 2 Prediction Data.csv')

In [5]:
def hyperbolic(x):
    return (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))

def generate_neurons_uniform(X,num_neurons):
    k = X.shape[1]
    variances = np.std(X,axis=0)
    weights = []
    for i in range(k):
        λ = variances[i]/np.sum(variances)
        a = 2.5 * λ / np.max(np.abs(X[:,i]))
        weight = np.random.uniform(low=-a,high=a,size=num_neurons)
        weights.append(weight)
    weights = np.array(weights)
    intercept = np.random.uniform(low=-1,high=1,size=num_neurons)
    
    return weights, intercept

def cal_neuron_val(X, weights, intercept):
    neuron_val = np.matmul(X,weights) + intercept
    neuron_val = hyperbolic(neuron_val)
    return neuron_val


In [6]:
def mbelm(data,coordinate,num_neurons=50):
    weights, intercept = generate_neurons_uniform(data[['Mean Curvature','Mean Angle']].values,num_neurons=num_neurons)
    neuron_val = cal_neuron_val(data[['Mean Curvature','Mean Angle']].values,weights,intercept)
    clf = BayesianRidge()
    name = 'Pointwise Variance ' + coordinate
    clf.fit(neuron_val, data[name].values)
    return {'weight':weights,'intercept':intercept,'model':clf}

In [7]:
def mbelm_pred(train,model):
    neuron_pred = cal_neuron_val(train[['curvature','angle']].values, model['weight'],model['intercept'])
    return model['model'].predict(neuron_pred)

In [8]:
convex_x = mbelm(convex,'X')
convex_y = mbelm(convex,'Y')
convex_z = mbelm(convex,'Z')

freeform_x = mbelm(freeform,'X')
freeform_y = mbelm(freeform,'Y')
freeform_z = mbelm(freeform,'Z')

In [9]:
def procrustes_distance(landmarks, mean):
    sum_dis = 0
    for i in range(len(landmarks)):
        sum_dis += np.linalg.norm(landmarks[i][['X','Y','Z']]-mean) ** 2
    return sum_dis

def opr(X_1, X_2):
    # Ordinary Procrustes registration involve only rotation.
    # Rotate X_1 onto X_2
    reg_mat = np.matmul(X_2.T, X_1) / (np.linalg.norm(X_1)*np.linalg.norm(X_2))
    V, Λ, U_t = np.linalg.svd(reg_mat)
    Γ = np.matmul(U_t.T, V.T)
    return Γ

def opa(X_1,mean):
    Γ = opr(X_1[['X','Y','Z']].values, mean[['X','Y','Z']].values)
    X_1[['X','Y','Z']] = np.matmul(X_1[['X','Y','Z']], Γ)
    return X_1

def ppa(landmarks):
    # Partial Procrustes registration algorithm for landmarks.
    
    ## Step 1: Center the landmarks
    size = len(landmarks[0])
    for i in range(len(landmarks)):
        x = np.mean(landmarks[i]['X'])
        y = np.mean(landmarks[i]['Y'])
        z = np.mean(landmarks[i]['Z'])
        landmarks[i][['X', 'Y', 'Z']] = landmarks[i][['X', 'Y', 'Z']] - np.array([x, y, z])

        
    
    ## Step 2: Initialize μ
    for i in range(len(landmarks)):
        if i == 0:
            sum_pts = landmarks[0][['X','Y','Z']]
        else:
            sum_pts += landmarks[i][['X', 'Y','Z']]
            
    μ = sum_pts / len(landmarks)
    
    ## Step 3: Remove rotation
    step = 0

    test = True
    while test:
        step+=1
#         print(step)
        distance = procrustes_distance(landmarks,μ)
        
        for i in range(len(landmarks)):
            Γ = opr(landmarks[i][['X','Y','Z']].values, μ.values)
            landmarks[i][['X','Y','Z']] = np.matmul(landmarks[i][['X','Y','Z']], Γ)
            landmarks[i][['nX','nY','nZ']] = np.matmul(landmarks[i][['nX','nY','nZ']],Γ)
        
        for i in range(len(landmarks)):
            if i == 0:
                sum_pts = landmarks[0][['X','Y','Z']]
                sum_normal = landmarks[0][['nX','nY','nZ']]
            else:
                sum_pts += landmarks[i][['X', 'Y','Z']]
                sum_normal += landmarks[i][['nX','nY','nZ']]
            
        μ = sum_pts / len(landmarks)
        μ_normal = sum_normal / len(landmarks)
        
        temp_dist = procrustes_distance(landmarks, μ)
        if (abs(distance - temp_dist) < 0.0001):
            test = False
    
    mean = pd.DataFrame()
    mean[['X','Y','Z']] = μ[['X','Y','Z']]
    mean[['nX','nY','nZ']] = μ_normal[['nX','nY','nZ']]

    return landmarks, mean
    
def means(lms):
    for i in range(len(lms)):
        if i == 0:
            sum_total = lms[i][['curvature','angle','Z']]
        else:
            sum_total += lms[i][['curvature','angle','Z']]
    return sum_total/len(lms)



In [14]:
########### ball ###########
landmark = pd.read_csv("ball landmarks.csv")

# generating bootstrap samples
X_var = mbelm_pred(landmark,convex_x)
Y_var = mbelm_pred(landmark,convex_y)
Z_var = mbelm_pred(landmark,convex_z)
var_X = np.array([np.random.normal(loc=0,scale=np.sqrt(np.abs(X_var[q])),size=1000) for q in range(len(X_var))])
var_Y = np.array([np.random.normal(loc=0,scale=np.sqrt(np.abs(Y_var[q])),size=1000) for q in range(len(Y_var))])
var_Z = np.array([np.random.normal(loc=0,scale=np.sqrt(np.abs(Z_var[q])),size=1000) for q in range(len(Z_var))])
boot_sample = []
for j in range(1000):
    this_loop = pd.DataFrame({'X':landmark['X'].values+var_X[:,j],'Y':landmark['Y'].values+var_Y[:,j],'Z':landmark['Z'].values+var_Z[:,j],'nX':landmark['nX'].values,'nY':landmark['nY'].values,'nZ':landmark['nZ'].values})
    boot_sample.append(this_loop)

In [15]:
# one scan case
scan_1_max_dev = []
scan_1_mean_dev = []

for boot_length in range(1000):
    selection = np.random.randint(0,1000)
    mean = landmark
    reg_conf,_ = ppa([boot_sample[selection].copy(),mean.copy()])
    dev = reg_conf[0][['X','Y','Z']] - reg_conf[1][['X','Y','Z']]
    devs = np.abs(np.sum(dev.values*reg_conf[1][['nX','nY','nZ']].values, axis=1))
    scan_1_max_dev.append(np.max(devs))
    scan_1_mean_dev.append(np.mean(devs))

quantiles = pd.DataFrame({"Maximum Deviation":scan_1_max_dev,"Mean Deviation":scan_1_mean_dev})
pickling_on = open("ball result/Ball " + str(1) + " scans.pickle","wb")
pickle.dump(quantiles, pickling_on)
pickling_on.close()



In [16]:
# multiple scan case
size = list(range(2,7))

for s in size:
    print("~~~~~~~~~~~~ Scan",s,"~~~~~~~~~~~~~~")
    scan_max_dev = []
    scan_mean_dev = []
    combs = []
    for i in range(2000):
        this_l = []
        for j in range(s):
            this_l.append(np.random.randint(0,1000))
        combs.append(this_l)
    choice = np.random.choice(list(range(len(combs))),size=1000,replace=False)

    for c in range(len(choice)):
        
        this_boot_landmarks = [boot_sample[i] for i in combs[choice[c]]]
        _, mean_conf = ppa(this_boot_landmarks)
        mean = landmark
        reg_conf,_ = ppa([mean_conf.copy(),mean.copy()])
        dev = reg_conf[0][['X','Y','Z']] - reg_conf[1][['X','Y','Z']]
        devs = np.abs(np.sum(dev.values*reg_conf[1][['nX','nY','nZ']].values, axis=1))
        scan_max_dev.append(np.max(devs))
        scan_mean_dev.append(np.mean(devs))
        
    results = pd.DataFrame({"Maximum Deviation":scan_max_dev,"Mean Deviation":scan_mean_dev})
    pickling_on = open("ball result/Ball " + str(s) + " scans.pickle","wb")
    pickle.dump(results, pickling_on)
    pickling_on.close()
        

~~~~~~~~~~~~ Scan 2 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 3 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 4 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 5 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 6 ~~~~~~~~~~~~~~


In [10]:
########### convex ###########
landmark = pd.read_csv("convex landmarks.csv")

# generating bootstrap samples
X_var = mbelm_pred(landmark,freeform_x)
Y_var = mbelm_pred(landmark,freeform_y)
Z_var = mbelm_pred(landmark,freeform_z)
var_X = np.array([np.random.normal(loc=0,scale=np.sqrt(np.abs(X_var[q])),size=1000) for q in range(len(X_var))])
var_Y = np.array([np.random.normal(loc=0,scale=np.sqrt(np.abs(Y_var[q])),size=1000) for q in range(len(Y_var))])
var_Z = np.array([np.random.normal(loc=0,scale=np.sqrt(np.abs(Z_var[q])),size=1000) for q in range(len(Z_var))])
boot_sample = []
for j in range(1000):
    this_loop = pd.DataFrame({'X':landmark['X'].values+var_X[:,j],'Y':landmark['Y'].values+var_Y[:,j],'Z':landmark['Z'].values+var_Z[:,j],'nX':landmark['nX'].values,'nY':landmark['nY'].values,'nZ':landmark['nZ'].values})
    boot_sample.append(this_loop)

In [11]:
# one scan case
scan_1_max_dev = []
scan_1_mean_dev = []

for boot_length in range(1000):
    selection = np.random.randint(0,1000)
    mean = landmark
    reg_conf,_ = ppa([boot_sample[selection].copy(),mean.copy()])
    dev = reg_conf[0][['X','Y','Z']] - reg_conf[1][['X','Y','Z']]
    devs = np.abs(np.sum(dev.values*reg_conf[1][['nX','nY','nZ']].values, axis=1))
    scan_1_max_dev.append(np.max(devs))
    scan_1_mean_dev.append(np.mean(devs))

quantiles = pd.DataFrame({"Maximum Deviation":scan_1_max_dev,"Mean Deviation":scan_1_mean_dev})
pickling_on = open("convex result/Convex " + str(1) + " scans.pickle","wb")
pickle.dump(quantiles, pickling_on)
pickling_on.close()

# multiple scan case
size = list(range(2,21))

for s in size:
    print("~~~~~~~~~~~~ Scan",s,"~~~~~~~~~~~~~~")
    scan_max_dev = []
    scan_mean_dev = []
    combs = []
    for i in range(2000):
        this_l = []
        for j in range(s):
            this_l.append(np.random.randint(0,1000))
        combs.append(this_l)
    choice = np.random.choice(list(range(len(combs))),size=1000,replace=False)

    for c in range(len(choice)):
        
        this_boot_landmarks = [boot_sample[i] for i in combs[choice[c]]]
        _, mean_conf = ppa(this_boot_landmarks)
        mean = landmark
        reg_conf,_ = ppa([mean_conf.copy(),mean.copy()])
        dev = reg_conf[0][['X','Y','Z']] - reg_conf[1][['X','Y','Z']]
        devs = np.abs(np.sum(dev.values*reg_conf[1][['nX','nY','nZ']].values, axis=1))
        scan_max_dev.append(np.max(devs))
        scan_mean_dev.append(np.mean(devs))
        
    results = pd.DataFrame({"Maximum Deviation":scan_max_dev,"Mean Deviation":scan_mean_dev})
    pickling_on = open("convex result/Convex " + str(s) + " scans.pickle","wb")
    pickle.dump(results, pickling_on)
    pickling_on.close()
        

~~~~~~~~~~~~ Scan 2 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 3 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 4 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 5 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 6 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 7 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 8 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 9 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 10 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 11 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 12 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 13 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 14 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 15 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 16 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 17 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 18 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 19 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 20 ~~~~~~~~~~~~~~


In [12]:
########### freeform ###########
landmark = pd.read_csv("freeform landmarks.csv")

# generating bootstrap samples
X_var = mbelm_pred(landmark,convex_x)
Y_var = mbelm_pred(landmark,convex_y)
Z_var = mbelm_pred(landmark,convex_z)
var_X = np.array([np.random.normal(loc=0,scale=np.sqrt(np.abs(X_var[q])),size=1000) for q in range(len(X_var))])
var_Y = np.array([np.random.normal(loc=0,scale=np.sqrt(np.abs(Y_var[q])),size=1000) for q in range(len(Y_var))])
var_Z = np.array([np.random.normal(loc=0,scale=np.sqrt(np.abs(Z_var[q])),size=1000) for q in range(len(Z_var))])
boot_sample = []
for j in range(1000):
    this_loop = pd.DataFrame({'X':landmark['X'].values+var_X[:,j],'Y':landmark['Y'].values+var_Y[:,j],'Z':landmark['Z'].values+var_Z[:,j],'nX':landmark['nX'].values,'nY':landmark['nY'].values,'nZ':landmark['nZ'].values})
    boot_sample.append(this_loop)

In [13]:
# one scan case
scan_1_max_dev = []
scan_1_mean_dev = []

for boot_length in range(1000):
    selection = np.random.randint(0,1000)
    mean = landmark
    reg_conf,_ = ppa([boot_sample[selection].copy(),mean.copy()])
    dev = reg_conf[0][['X','Y','Z']] - reg_conf[1][['X','Y','Z']]
    devs = np.abs(np.sum(dev.values*reg_conf[1][['nX','nY','nZ']].values, axis=1))
    scan_1_max_dev.append(np.max(devs))
    scan_1_mean_dev.append(np.mean(devs))

quantiles = pd.DataFrame({"Maximum Deviation":scan_1_max_dev,"Mean Deviation":scan_1_mean_dev})
pickling_on = open("freeform result/Freeform " + str(1) + " scans.pickle","wb")
pickle.dump(quantiles, pickling_on)
pickling_on.close()

# multiple scan case
size = list(range(2,21))

for s in size:
    print("~~~~~~~~~~~~ Scan",s,"~~~~~~~~~~~~~~")
    scan_max_dev = []
    scan_mean_dev = []
    combs = []
    for i in range(2000):
        this_l = []
        for j in range(s):
            this_l.append(np.random.randint(0,1000))
        combs.append(this_l)
    choice = np.random.choice(list(range(len(combs))),size=1000,replace=False)

    for c in range(len(choice)):
        
        this_boot_landmarks = [boot_sample[i] for i in combs[choice[c]]]
        _, mean_conf = ppa(this_boot_landmarks)
        mean = landmark
        reg_conf,_ = ppa([mean_conf.copy(),mean.copy()])
        dev = reg_conf[0][['X','Y','Z']] - reg_conf[1][['X','Y','Z']]
        devs = np.abs(np.sum(dev.values*reg_conf[1][['nX','nY','nZ']].values, axis=1))
        scan_max_dev.append(np.max(devs))
        scan_mean_dev.append(np.mean(devs))
        
    results = pd.DataFrame({"Maximum Deviation":scan_max_dev,"Mean Deviation":scan_mean_dev})
    pickling_on = open("freeform result/Freeform " + str(s) + " scans.pickle","wb")
    pickle.dump(results, pickling_on)
    pickling_on.close()
        

~~~~~~~~~~~~ Scan 2 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 3 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 4 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 5 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 6 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 7 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 8 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 9 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 10 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 11 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 12 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 13 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 14 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 15 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 16 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 17 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 18 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 19 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 20 ~~~~~~~~~~~~~~


In [14]:
########### freeform 2 ###########
landmark = pd.read_csv("freeform 2 landmarks.csv")

# generating bootstrap samples
X_var = mbelm_pred(landmark,convex_x)
Y_var = mbelm_pred(landmark,convex_y)
Z_var = mbelm_pred(landmark,convex_z)
var_X = np.array([np.random.normal(loc=0,scale=np.sqrt(np.abs(X_var[q])),size=1000) for q in range(len(X_var))])
var_Y = np.array([np.random.normal(loc=0,scale=np.sqrt(np.abs(Y_var[q])),size=1000) for q in range(len(Y_var))])
var_Z = np.array([np.random.normal(loc=0,scale=np.sqrt(np.abs(Z_var[q])),size=1000) for q in range(len(Z_var))])
boot_sample = []
for j in range(1000):
    this_loop = pd.DataFrame({'X':landmark['X'].values+var_X[:,j],'Y':landmark['Y'].values+var_Y[:,j],'Z':landmark['Z'].values+var_Z[:,j],'nX':landmark['nX'].values,'nY':landmark['nY'].values,'nZ':landmark['nZ'].values})
    boot_sample.append(this_loop)

In [15]:
# one scan case
scan_1_max_dev = []
scan_1_mean_dev = []

for boot_length in range(1000):
    selection = np.random.randint(0,1000)
    mean = landmark
    reg_conf,_ = ppa([boot_sample[selection].copy(),mean.copy()])
    dev = reg_conf[0][['X','Y','Z']] - reg_conf[1][['X','Y','Z']]
    devs = np.abs(np.sum(dev.values*reg_conf[1][['nX','nY','nZ']].values, axis=1))
    scan_1_max_dev.append(np.max(devs))
    scan_1_mean_dev.append(np.mean(devs))

quantiles = pd.DataFrame({"Maximum Deviation":scan_1_max_dev,"Mean Deviation":scan_1_mean_dev})
pickling_on = open("freeform 2 result/Freeform 2 " + str(1) + " scans.pickle","wb")
pickle.dump(quantiles, pickling_on)
pickling_on.close()

# multiple scan case
size = list(range(2,21))

for s in size:
    print("~~~~~~~~~~~~ Scan",s,"~~~~~~~~~~~~~~")
    scan_max_dev = []
    scan_mean_dev = []
    combs = []
    for i in range(2000):
        this_l = []
        for j in range(s):
            this_l.append(np.random.randint(0,1000))
        combs.append(this_l)
    choice = np.random.choice(list(range(len(combs))),size=1000,replace=False)

    for c in range(len(choice)):
        
        this_boot_landmarks = [boot_sample[i] for i in combs[choice[c]]]
        _, mean_conf = ppa(this_boot_landmarks)
        mean = landmark
        reg_conf,_ = ppa([mean_conf.copy(),mean.copy()])
        dev = reg_conf[0][['X','Y','Z']] - reg_conf[1][['X','Y','Z']]
        devs = np.abs(np.sum(dev.values*reg_conf[1][['nX','nY','nZ']].values, axis=1))
        scan_max_dev.append(np.max(devs))
        scan_mean_dev.append(np.mean(devs))
        
    results = pd.DataFrame({"Maximum Deviation":scan_max_dev,"Mean Deviation":scan_mean_dev})
    pickling_on = open("freeform 2 result/Freeform 2 " + str(s) + " scans.pickle","wb")
    pickle.dump(results, pickling_on)
    pickling_on.close()
        

~~~~~~~~~~~~ Scan 2 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 3 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 4 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 5 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 6 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 7 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 8 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 9 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 10 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 11 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 12 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 13 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 14 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 15 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 16 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 17 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 18 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 19 ~~~~~~~~~~~~~~
~~~~~~~~~~~~ Scan 20 ~~~~~~~~~~~~~~
